In [9]:
import tkinter as tk
from tkinter import messagebox, Toplevel, Listbox, END
from math import sin, cos, tan, log, sqrt, e, pi

class MultiModeCalculator(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Multi-Mode Calculator")
        self.resizable(False, False)
        
        # Theme settings
        self.dark_mode = False
        self.bg_color = "#ffffff"
        self.fg_color = "#000000"
        self.btn_bg = "#f0f0f0"
        self.btn_fg = "#000000"
        
        # History list: will store tuples of (expression, result)
        # For APY mode, we might store ("APY: principal=..., rate=..., comp=..., years=...", result)
        self.history = []
        
        # Current mode: 'basic', 'scientific', or 'apy'
        self.current_mode = tk.StringVar(value="scientific")
        
        # Create a menu bar
        self.menu_bar = tk.Menu(self)
        self.config(menu=self.menu_bar)
        
        # File menu
        file_menu = tk.Menu(self.menu_bar, tearoff=0)
        file_menu.add_command(label="Exit", command=self.quit)
        self.menu_bar.add_cascade(label="File", menu=file_menu)
        
        # View menu
        view_menu = tk.Menu(self.menu_bar, tearoff=0)
        view_menu.add_command(label="Toggle Dark Mode", command=self.toggle_dark_mode)
        view_menu.add_command(label="View History", command=self.view_history)
        
        # Mode submenu
        mode_menu = tk.Menu(view_menu, tearoff=0)
        mode_menu.add_radiobutton(label="Basic", variable=self.current_mode, value="basic", command=self.set_mode)
        mode_menu.add_radiobutton(label="Scientific", variable=self.current_mode, value="scientific", command=self.set_mode)
        mode_menu.add_radiobutton(label="APY", variable=self.current_mode, value="apy", command=self.set_mode)
        view_menu.add_cascade(label="Calculator Mode", menu=mode_menu)
        
        self.menu_bar.add_cascade(label="View", menu=view_menu)
        
        # Help menu
        help_menu = tk.Menu(self.menu_bar, tearoff=0)
        help_menu.add_command(label="About", command=self.show_about)
        self.menu_bar.add_cascade(label="Help", menu=help_menu)
        
        # Entry widget for basic/scientific modes
        # For APY mode, we'll hide or repurpose the layout.
        self.expression_var = tk.StringVar()
        self.entry = tk.Entry(self, textvariable=self.expression_var, font=('Arial', 18), bd=10, 
                              insertwidth=2, width=24, borderwidth=0)
        
        # We'll build the UI according to the current mode
        self.build_ui("scientific")
        
        self.update_theme()  # Set initial theme colors
        
    def build_ui(self, mode):
        # Clear the window except the menu
        for widget in self.grid_slaves():
            widget.destroy()

        if mode in ["basic", "scientific"]:
            # Show the entry for expressions
            self.entry = tk.Entry(self, textvariable=self.expression_var, font=('Arial', 18), bd=10, 
                                  insertwidth=2, width=24, borderwidth=0)
            self.entry.grid(row=0, column=0, columnspan=6, sticky="we", padx=5, pady=5)
        
        if mode == "basic":
            buttons = [
                ['7', '8', '9', '/'],
                ['4', '5', '6', '*'],
                ['1', '2', '3', '-'],
                ['0', '.', '=', '+'],
                ['C']
            ]
            self.build_buttons(buttons)
        
        elif mode == "scientific":
            buttons = [
                ['7', '8', '9', '/', 'sin', 'cos'],
                ['4', '5', '6', '*', 'tan', 'log'],
                ['1', '2', '3', '-', '√', '^'],
                ['0', '.', '(', ')', '+', '='],
                ['C', 'π', 'e']
            ]
            self.build_buttons(buttons)
        
        elif mode == "apy":
            # In APY mode, we show fields instead of a keypad
            # Destroy the entry if it exists, as APY mode does not use the main entry in the same way
            # Instead of the expression entry, we will use separate labels and entries
            self.build_apy_interface()
        
        # Reapply theme
        self.update_theme()
        
    def build_buttons(self, buttons):
        # Create the buttons for basic/scientific
        start_row = 1
        for r, row in enumerate(buttons, start=start_row):
            for c, btn_text in enumerate(row):
                action = lambda x=btn_text: self.on_button_click(x)
                btn = tk.Button(self, text=btn_text, width=5, height=2, font=('Arial', 14), 
                                command=action, bg=self.btn_bg, fg=self.btn_fg)
                btn.grid(row=r, column=c, padx=1, pady=1)
        
    def build_apy_interface(self):
        # Create APY-related labels and entries
        # We'll have entries for principal, annual interest rate, compounding, years
        # and buttons to calculate APY and future value
        tk.Label(self, text="Principal:", font=('Arial', 14), bg=self.bg_color, fg=self.fg_color).grid(row=0, column=0, sticky='e', padx=5, pady=5)
        self.principal_var = tk.StringVar()
        tk.Entry(self, textvariable=self.principal_var, font=('Arial', 14), width=10).grid(row=0, column=1, padx=5, pady=5)
        
        tk.Label(self, text="Annual Interest Rate (decimal):", font=('Arial', 14), bg=self.bg_color, fg=self.fg_color).grid(row=1, column=0, sticky='e', padx=5, pady=5)
        self.rate_var = tk.StringVar()
        tk.Entry(self, textvariable=self.rate_var, font=('Arial', 14), width=10).grid(row=1, column=1, padx=5, pady=5)
        
        tk.Label(self, text="Compounding/Year:", font=('Arial', 14), bg=self.bg_color, fg=self.fg_color).grid(row=2, column=0, sticky='e', padx=5, pady=5)
        self.comp_var = tk.StringVar()
        tk.Entry(self, textvariable=self.comp_var, font=('Arial', 14), width=10).grid(row=2, column=1, padx=5, pady=5)
        
        tk.Label(self, text="Years:", font=('Arial', 14), bg=self.bg_color, fg=self.fg_color).grid(row=3, column=0, sticky='e', padx=5, pady=5)
        self.years_var = tk.StringVar()
        tk.Entry(self, textvariable=self.years_var, font=('Arial', 14), width=10).grid(row=3, column=1, padx=5, pady=5)
        
        # Buttons for APY calculation and future value calculation
        tk.Button(self, text="Calculate APY", font=('Arial', 14), bg=self.btn_bg, fg=self.btn_fg, command=self.calculate_apy).grid(row=4, column=0, padx=5, pady=5)
        tk.Button(self, text="Calculate Future Value", font=('Arial', 14), bg=self.btn_bg, fg=self.btn_fg, command=self.calculate_future_value).grid(row=4, column=1, padx=5, pady=5)
        
        # A label to show the result
        self.apy_result_var = tk.StringVar(value="Result will be shown here.")
        tk.Label(self, textvariable=self.apy_result_var, font=('Arial', 14), bg=self.bg_color, fg=self.fg_color).grid(row=5, column=0, columnspan=2, pady=10)
        
    def on_button_click(self, char):
        mode = self.current_mode.get()
        if mode in ["basic", "scientific"]:
            if char == 'C':
                # Clear the entire expression
                self.expression_var.set("")
            elif char == '=':
                self.calculate_result()
            elif char == '√':
                # Insert sqrt(
                self.expression_var.set(self.expression_var.get() + "sqrt(")
            elif char == 'π':
                # Insert pi value
                self.expression_var.set(self.expression_var.get() + "pi")
            elif char in ['sin', 'cos', 'tan', 'log']:
                # Insert function + (
                self.expression_var.set(self.expression_var.get() + char + "(")
            elif char == '^':
                # Use ** for exponentiation
                self.expression_var.set(self.expression_var.get() + "**")
            else:
                # Just add the character to the expression
                self.expression_var.set(self.expression_var.get() + char)

    def calculate_result(self):
        # For basic/scientific modes
        expr = self.expression_var.get()
        try:
            result = eval(expr)
            self.expression_var.set(str(result))
            # Store calculation in history
            self.history.append((expr, str(result)))
        except Exception:
            self.expression_var.set("Error")
    
    def calculate_apy(self):
        # For APY mode
        try:
            principal = float(self.principal_var.get())
            rate = float(self.rate_var.get())
            comp = float(self.comp_var.get())
            years = float(self.years_var.get())
            
            # APY calculation: (1 + r/n)^n - 1
            apy_value = (1 + rate/comp)**comp - 1
            # Store in history
            calc_str = f"APY Calc: P={principal}, rate={rate}, comp={comp}, yrs={years} -> APY={apy_value:.4%}"
            self.history.append((calc_str, f"{apy_value:.4%}"))
            
            self.apy_result_var.set(f"APY: {apy_value:.4%}")
        except ValueError:
            self.apy_result_var.set("Error: Invalid input.")
    
    def calculate_future_value(self):
        # For APY mode, calculate future value: FV = P * (1 + r/n)^(n*years)
        try:
            principal = float(self.principal_var.get())
            rate = float(self.rate_var.get())
            comp = float(self.comp_var.get())
            years = float(self.years_var.get())
            
            fv = principal * (1 + rate/comp)**(comp*years)
            # Store in history
            calc_str = f"FV Calc: P={principal}, rate={rate}, comp={comp}, yrs={years} -> FV={fv:.2f}"
            self.history.append((calc_str, f"{fv:.2f}"))
            
            self.apy_result_var.set(f"Future Value: ${fv:.2f}")
        except ValueError:
            self.apy_result_var.set("Error: Invalid input.")

    def toggle_dark_mode(self):
        self.dark_mode = not self.dark_mode
        if self.dark_mode:
            self.bg_color = "#333333"
            self.fg_color = "#ffffff"
            self.btn_bg = "#555555"
            self.btn_fg = "#ffffff"
        else:
            self.bg_color = "#ffffff"
            self.fg_color = "#000000"
            self.btn_bg = "#f0f0f0"
            self.btn_fg = "#000000"
        self.update_theme()

    def update_theme(self):
        self.configure(bg=self.bg_color)
        
        # Update children widgets
        for child in self.winfo_children():
            # Buttons
            if isinstance(child, tk.Button):
                child.configure(bg=self.btn_bg, fg=self.btn_fg)
            # Entry
            elif isinstance(child, tk.Entry):
                child.configure(bg=self.bg_color, fg=self.fg_color, insertbackground=self.fg_color)
            # Labels
            elif isinstance(child, tk.Label):
                child.configure(bg=self.bg_color, fg=self.fg_color)

    def view_history(self):
        # Create a new window to show history
        hist_window = Toplevel(self)
        hist_window.title("Calculation History")
        hist_window.resizable(False, False)
        hist_list = Listbox(hist_window, width=60, height=10, font=('Arial', 12))
        hist_list.pack(padx=10, pady=10)
        
        # Populate history list
        for expr, res in self.history:
            hist_list.insert(END, f"{expr} = {res}")

    def show_about(self):
        messagebox.showinfo("About", "Multi-Mode Calculator\n- Basic\n- Scientific\n- APY Calculator\nCreated with Python and Tkinter")

    def set_mode(self):
        mode = self.current_mode.get()
        # Clear the expression if switching away from sci/basic
        if mode in ["basic", "scientific"]:
            self.expression_var.set("")
        self.build_ui(mode)

if __name__ == "__main__":
    app = MultiModeCalculator()
    app.mainloop()
